In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import json
from pymongo import MongoClient
from datetime import datetime
from dateutil.parser import parse

In [2]:
df = pd.read_excel("/home/waris/Github/tupras-analysis/new-alarm/proses_alarm.xls",skipinitialspace=True, usecols=["MachineName","SourceName","EventTime", "Message","Condition","Attributes"])

In [3]:
# initial processing
for i in range(0, len(df["EventTime"])):
    d = df["EventTime"][i]
    d = d.replace(".000000000","")
    d = d.replace("/","-")
    df["EventTime"][i] = parse(d)


In [4]:
print("Date-Type:", type(df["EventTime"][1]))
print(df)

Date-Type: <class 'datetime.datetime'>
      MachineName SourceName            EventTime  \
0          PHD47B  48TIC2026  2020-01-27 09:55:34   
1          PHD47B  48TIC2026  2020-01-27 09:55:36   
2          PHD47B  48TIC2026  2020-01-27 09:55:36   
3          PHD47B  48TIC2026  2020-01-27 09:55:38   
4          PHD47B  48TIC2026  2020-01-27 09:55:38   
...           ...        ...                  ...   
42089      PHD47B  48TIC2026  2020-01-27 19:05:33   
42090      PHD47B   47TI931A  2020-01-27 19:05:33   
42091      PHD47B   47TI002C  2020-01-27 19:05:34   
42092      PHD47B   47TI002C  2020-01-27 19:05:35   
42093      PHD47B   47TI002C  2020-01-27 19:05:35   

                                                 Message Condition  \
0      48TIC2026    E208 CKS    TEMIZ MDEA   PV  =   ...      VEL-   
1      48TIC2026    E208 CKS    TEMIZ MDEA   PV  =   ...      VEL-   
2      48TIC2026    E208 CKS    TEMIZ MDEA   PV  =   ...      VEL+   
3      48TIC2026    E208 CKS    TEMIZ MDEA  

In [5]:
# client = MongoClient()
# db = client["tupras-db"]
# collection_raw_alarm = db["alarm"]
# data = json.loads(df.to_json(orient='records'))
# collection_raw_alarm.remove()
# print(data)
# collection_raw_alarm.insert(data)

# Preprocessing of Alarm data

There are different types of alarm data but Excel file only contains "Process Alarm Data".

**Excel Columns Description**
* MachineName: It represent the unit (e.g.,Plant-47)
* SourceName: The names of the devices operating in a unit (e.g., controller, transmitter, and calculator block names)
    1. Five types of controllers: TIC, PIC, FIC, LIC, and AIC.
    2. Three types of transmitters: TI, FI, LI
    3. Calculator block names ?

* EventTime: The date when an alarm is appeared. It also represent when it recovered right?

**Deactivation of an Alarm:**
* Recover : The alram is deactivated.
* IOP  : Communication between the DCS and field is cut off (alarm activated).
* IOP + Recover: communication problem is solved and the alarm is deactivated. 
* ALM : It represent alarm is activated
* NR : The alarm is returned to normal state. 

**Questions:**
1. Before alarm data analysis, if ALM, NR and RECOVER messages are seen, these rows must be deleted. Why?
2. Alarm-off 
3. shelving of alarms
4. Can alarm cannot trigger twice? 
5. 

In [6]:
sources = {}
source_conditions = {}
conditions = {} # Vel, IOP ALM, NR etc
key = ""
conditions["Recover"] = 0
for i in range(0, len(df["EventTime"])):
    key = df["SourceName"][i]
    temp_dict = {"SourceName": key, "EventTime": df["EventTime"][i], "Message":df["Message"][i],"Condition": df["Condition"][i],"Attributes":df["Attributes"][i]}
    
    if sources.get(key) != None:
        sources[key].append(temp_dict)
        
        #source conditions
        d =  source_conditions[key]
        if d.get(temp_dict["Condition"]) == None:
            d[temp_dict["Condition"]] = 0    
        d[temp_dict["Condition"]]  += 1
        d["Total"] += 1
        
        if d.get("Recover") == None:
            d["Recover"] = 0
        
        # conditions           
        c = temp_dict["Condition"]
        if conditions.get(c) == None:  
            conditions[c] = 0
        conditions[c] +=1
        
        if temp_dict["Message"].find("Recover") != -1:
            conditions["Recover"] += 1
            d["Recover"] += 1
        if temp_dict["Message"].strip().find("NR") != -1:
            print(temp_dict["Message"])
            d["Recover"] +=1
            conditions["Recover"] += 1
    else:
        sources[key] = []
        source_conditions[key] = {}
        sources[key].append(temp_dict)
        d = source_conditions[key]
        d[temp_dict["Condition"]] = 1
        d["Total"] = 1
        
        if d.get("Recover") == None:
            d["Recover"] = 0
        
        c = temp_dict["Condition"]
        if conditions.get(c) == None:  
            conditions[c] = 0
        conditions[c] +=1
        
        if temp_dict["Message"].find("Recover") != -1:
            conditions["Recover"] += 1
            d["Recover"] +=1
        if temp_dict["Message"].strip().find("NR") != -1:
            print(temp_dict["Message"])
            d["Recover"] +=1
            conditions["Recover"] += 1


# sorting the source conditions dictionary based on the number of alarms related to each sensor
source_conditions = {k:v for k,v in sorted(source_conditions.items(), key=lambda item: item[1]["Total"],reverse=True)}
conditions = {k:v for k,v in sorted(conditions.items(), key=lambda item: item[1], reverse=True)}

47TAL003AB-ANN G-3 EMIS SICAKLIGI NR
47TAL003AB-ANN G-3 EMIS SICAKLIGI NR
47TAL003AB-ANN G-3 EMIS SICAKLIGI NR
47TAL003AB-ANN G-3 EMIS SICAKLIGI NR
47TAL003AB-ANN G-3 EMIS SICAKLIGI NR
47TAL003AB-ANN G-3 EMIS SICAKLIGI NR
47TAL003AB-ANN G-3 EMIS SICAKLIGI NR
47TAL003AB-ANN G-3 EMIS SICAKLIGI NR
47TAL003AB-ANN G-3 EMIS SICAKLIGI NR
47TAL003AB-ANN G-3 EMIS SICAKLIGI NR
47TAL003AB-ANN G-3 EMIS SICAKLIGI NR
48XL002-ANN  48G-1B STATUS            NR
47TAL003AB-ANN G-3 EMIS SICAKLIGI NR
47TAL003AB-ANN G-3 EMIS SICAKLIGI NR
47FSL019-ANN PDI-014 LO KOLUNA HIDRJN NR
47FSL019-ANN PDI-014 LO KOLUNA HIDRJN NR
47FSL019-ANN PDI-014 LO KOLUNA HIDRJN NR
47TAL003AB-ANN G-3 EMIS SICAKLIGI NR
47TAL003AB-ANN G-3 EMIS SICAKLIGI NR
47FD-032H-AN C-109 MERDIV ALEV YUK    NR
47FD-033F-AN E-113 ALEV ARIZA         NR
47FD-032HH-A C-109 MERDIV ALEV YUKYUK NR
47FD-032H-AN C-109 MERDIV ALEV YUK    NR
47FD-032HH-A C-109 MERDIV ALEV YUKYUK NR
47UAL801-ANN K-2 C MEKANIK LUBRIKATOR NR
47FSL019-ANN PDI-014 LO KOLUNA HIDR

In [7]:
# Total number of disctinct SourceNames in the excel sheet.

print("Total Number of Sourcenames: {s}\n\nSources List \n\n {l}".format(s=len(sources.keys()), l = sources.keys()))

Total Number of Sourcenames: 109

Sources List 

 dict_keys(['48TIC2026', '47TI931A', '47TAL003AB-ANN', '47FLI-032', '47TI002C', '47AI1503', '47TI003A', '47PIC1755', '47PDI1746', '47TI1507', '47TIC1511', '47TI3519', '47TIC3520', '47TI003B', '47XL015-ANN', '47XL016-ANN', '47LI003', '47TI2076', '48XL002-ANN', '47FSL019-ANN', '47TI1769', '47TI1506', '47LIC1509', 'SIO-18XA001', '47FI1005', '47TI2086', '47FIC021', '47TI3102', '47AIT2011', '47FLI-042', '47TI1512B', '47TI1512A', '47FD-032H-ANN', '47FD-032HH-ANN', '47FLI-039', '47FD-033F-ANN', '47TI1713', '47UAL801-ANN', '48XL006-ANN', '47PDI003', '47FLI-022', '47FIC1524_CALX', '48PAL013-ANN', '48XL004-ANN', '47LIC005', '47FD-036H-ANN', '47FD-036HH-ANN', '47FLI-036', '47LIC1501', '47FIC1563', '47TI2003', '47FIC037', '47TI1512ABC', '47TI1529', '47LIC1555', '47TI1512C', '47FI1568', '47TI2083', '47TI414O', '47TI416', '48LIC009', '47FIC3102', '47PIC1505', '48PI2025', '47FIC1522A', '48FIX008', '48AIC001', '47FLI-041', '47FLI-037', '47LIC3408', '48H

In [8]:
# How many times each SourceName has been triggered (activation + Deactivation)?
total  = 0
i =0
for key in source_conditions.keys():
    s1 = "Source: " + key + ", Total: {} | ".format(source_conditions[key]["Total"])
    s2 = ""
    total += source_conditions[key]["Total"]
    for k in source_conditions[key].keys():
        if k == "Total" or k == "Recover":
            continue
        s2 = s2 + k +" "+ str(source_conditions[key][k]) + ", "    
    s1 = s1 + s2
    print("{}. ".format(i),s1)
    i +=1
    if i == 10:
        break
print("\n\n Total Number of Entries in the excel:", total)
assert(total== len(df["SourceName"]))

0.  Source: 48TIC2026, Total: 25220 | VEL- 11675, VEL+ 13545, 
1.  Source: 47TI931A, Total: 10980 | IOP 10980, 
2.  Source: 47FLI-032, Total: 2395 | CNF 1176, IOP 1217, IOP- 2, 
3.  Source: 47AI1503, Total: 1460 | LO 1460, 
4.  Source: 47TI002C, Total: 668 | LLL 668, 
5.  Source: 47PDI1746, Total: 264 | IOP 264, 
6.  Source: 47FSL019-ANN, Total: 181 | ALM 181, 
7.  Source: 47TI003A, Total: 159 | LTRP 159, 
8.  Source: 47TI003B, Total: 84 | LTRP 84, 
9.  Source: 47TAL003AB-ANN, Total: 65 | ALM 65, 


 Total Number of Entries in the excel: 41476


AssertionError: 

In [9]:
i = 0
for key in source_conditions.keys():
    d = source_conditions[key]  
    print("{}. Source: {}, # of activations= {}, # of deactivations = {}, Diff = {}".format(i,key, d["Total"]-d["Recover"], d["Recover"], (d["Total"]-d["Recover"])-d["Recover"]))
    i += 1
    if i == 10:
        break

0. Source: 48TIC2026, # of activations= 12605, # of deactivations = 12615, Diff = -10
1. Source: 47TI931A, # of activations= 5492, # of deactivations = 5488, Diff = 4
2. Source: 47FLI-032, # of activations= 1250, # of deactivations = 1145, Diff = 105
3. Source: 47AI1503, # of activations= 740, # of deactivations = 720, Diff = 20
4. Source: 47TI002C, # of activations= 336, # of deactivations = 332, Diff = 4
5. Source: 47PDI1746, # of activations= 133, # of deactivations = 131, Diff = 2
6. Source: 47FSL019-ANN, # of activations= 98, # of deactivations = 83, Diff = 15
7. Source: 47TI003A, # of activations= 84, # of deactivations = 75, Diff = 9
8. Source: 47TI003B, # of activations= 43, # of deactivations = 41, Diff = 2
9. Source: 47TAL003AB-ANN, # of activations= 33, # of deactivations = 32, Diff = 1


In [10]:
# number of times each condition appear
for k in conditions.keys():
    if k == "Recover":
        continue
    print(k,conditions[k])

VEL+ 13545
IOP 12574
VEL- 11676
LO 1499
CNF 1220
LLL 672
ALM 382
LTRP 257
HI 117
HHH 60
ANS- 30
HTRP 21
HH 19
DV+ 11
LL 4
IOP- 4
DV- 3


In [11]:
print("Recover", conditions["Recover"])

Recover 20894


### Which message is related to which message (i.e., activation and deactivation) ?
Dispalying top 10 **SourceName** with 30 alarms in sorted order by **EventTime**

In [12]:
# conditions = {k:v for k,v in sorted(conditions.items(), key=lambda item: item[1], reverse=True)}

i = 0
for sname in source_conditions.keys():
    
    print("=======Source {}: {}========".format(i,sname))
    single_source_alarms = sources[sname]
    single_source_alarms = [v for v in sorted(single_source_alarms, key=lambda arg: arg["EventTime"], reverse = False) ]
    
    j = 0
    for alarm in single_source_alarms:
        print("{},---,{},--- Message: {}".format(alarm["EventTime"],alarm["Condition"], alarm["Message"]))
        j += 1
        if j == 30:
            break
    i += 1
    if i == 10:
        break


=======Source 0: 48TIC2026========
2020-01-27 09:36:35,---,VEL-,--- Message: 48TIC2026    E208 CKS    TEMIZ MDEA   PV  =   -4.4 C    VEL-
2020-01-27 09:36:38,---,VEL+,--- Message: 48TIC2026    E208 CKS    TEMIZ MDEA   PV  =   -4.1 C    VEL+ Recover
2020-01-27 09:36:45,---,VEL-,--- Message: 48TIC2026    E208 CKS    TEMIZ MDEA   PV  =   -4.1 C    VEL- Recover
2020-01-27 09:36:45,---,VEL+,--- Message: 48TIC2026    E208 CKS    TEMIZ MDEA   PV  =   -4.1 C    VEL+
2020-01-27 09:36:47,---,VEL+,--- Message: 48TIC2026    E208 CKS    TEMIZ MDEA   PV  =   -4.8 C    VEL+ Recover
2020-01-27 09:36:47,---,VEL-,--- Message: 48TIC2026    E208 CKS    TEMIZ MDEA   PV  =   -4.8 C    VEL-
2020-01-27 09:36:48,---,VEL-,--- Message: 48TIC2026    E208 CKS    TEMIZ MDEA   PV  =   -4.3 C    VEL- Recover
2020-01-27 09:36:48,---,VEL+,--- Message: 48TIC2026    E208 CKS    TEMIZ MDEA   PV  =   -4.3 C    VEL+
2020-01-27 09:36:52,---,VEL-,--- Message: 48TIC2026    E208 CKS    TEMIZ MDEA   PV  =   -4.3 C    VEL- Recove

**Find the duration of activation of each Alarm i.e., duration =  DeactivationTime - ActivationTime**

Compute mean, median, mode, on the duration... 


In [40]:
def activationAndDeactivation(alarms):
    queue = []
    alarms_with_sart_end_time = []
    alarms = [v for v in sorted(alarms, key=lambda arg: arg["EventTime"], reverse = False) ]
    for alarm in alarms:
#         print(alarm["Message"])
        if alarm["Message"].find("Recover") == -1:
            queue.append(alarm) # enqueue the activation
        else:
            if len(queue)== 0:
                print(alarm["Message"])
                continue
            d = queue.pop(0) # dequeue
            d = {k:v for k,v in d.items()}
            d["StartTime"] = d["EventTime"]
            d["EndTime"] = alarm["EventTime"]
            del d["EventTime"]
            
            alarms_with_sart_end_time.append(d)
            
    return alarms_with_sart_end_time

sname = "48TIC2026"
alarms = sources[sname]
alarms = activationAndDeactivation(alarms)
stats = [] 
for alarm in alarms:
    stats.append(alarm["EndTime"]-alarm["StartTime"])

stats = [v  for v in sorted(stats)]

print(stats[-1])


import statistics

# print(statistics.median(stats), statistics.mode(stats))

# print(stats[0], stats[-1])

print("min = {}, max = {}, median = {}, mode = {}, mean = ...".format(stats[0],stats[-1], statistics.median(stats), statistics.mode(stats)))

# print(statistics.mean(stats))

48TIC2026    E208 CKS    TEMIZ MDEA   PV  =   -4.1 C    VEL- Recover
48TIC2026    E208 CKS    TEMIZ MDEA   PV  =   -4.4 C    VEL+ Recover
48TIC2026    E208 CKS    TEMIZ MDEA   PV  =   -4.1 C    VEL- Recover
48TIC2026    E208 CKS    TEMIZ MDEA   PV  =   -4.1 C    VEL+ Recover
48TIC2026    E208 CKS    TEMIZ MDEA   PV  =   -4.3 C    VEL- Recover
48TIC2026    E208 CKS    TEMIZ MDEA   PV  =   -4.5 C    VEL+ Recover
48TIC2026    E208 CKS    TEMIZ MDEA   PV  =   -4.2 C    VEL- Recover
48TIC2026    E208 CKS    TEMIZ MDEA   PV  =   -4.2 C    VEL- Recover
48TIC2026    E208 CKS    TEMIZ MDEA   PV  =   -4.5 C    VEL- Recover
48TIC2026    E208 CKS    TEMIZ MDEA   PV  =   -4.2 C    VEL+ Recover


AttributeError: 'datetime.timedelta' object has no attribute 'second'